# The First Day of the Satellite

The first day of a satellite in orbit requires a lot of planning (even though the plans go out of the window rather quickly in many cases). The critical inputs to this planning are:
- When will we be able to talk to the satellite (groundstation contact times)
- Where should I point my groundstation antenna (groundstation antenna azimuth-elevation angles)
- When will the satellite see the sun and generate power (satellite illumination times)

This how-to guide shows how these bits of crucial information can be generated with satkit. The steps are:
1. Set up the orbit
2. Set up the groundstation
3. Set up the search interval and the propagator
4. Run the analyses

However, as always, the very first thing to do is to initialise satkit and Orekit.

In [2]:
from pathlib import Path

# If satkit import fails, try to locate the module
import os

try:
    import satkit
except ModuleNotFoundError:
    os.chdir(os.path.join("..", ".."))
    os.getcwd()

from satkit import init_satkit, u

init_satkit(Path("data", "orekit-data", "orekit-data-reference.zip"), Path("..", ".."))

# Orekit / satkit init complete

WindowsPath('C:/Users/EgemenImre/PycharmProjects/satkit/data/orekit-data/orekit-data-reference.zip')

Then the groundstation location is initialised. The `QuantityInit` class provides the `geodetic_location` method to initialise the groundstation location safely with units.

In [3]:
from satkit.utils.quantity_init import QuantityInit

# Set the groundstation location (geodetic coordinates)
longitude = 45.0 * u.deg
latitude = 40 * u.deg
altitude = 0 * u.m
gnd_station = QuantityInit.geodetic_location(latitude, longitude, altitude)

The next step is to set up the search interval. The search start start and search duration parameters are defined by the target analysis duration and are to be set by the user.

It should be noted that, if the initial coordinates are defined in cartesian (for example when the separation coordinates from the launcher are known), a different propagator with a more detailed force model can be used. This is set with the 

In [4]:
from org.orekit.frames import FramesFactory
from org.orekit.time import TimeScalesFactory

from satkit.time.time import AbsoluteDateExt
from satkit.time.timeinterval import TimeInterval

# Shorthand for UTC
utc = TimeScalesFactory.getUTC()

# Set user-defined search interval
search_start = AbsoluteDateExt("2020-06-10T00:00:00.000", utc)
duration = 1 * u.day
search_interval = TimeInterval.from_duration(search_start, duration)


Finally the orbit and propagator are set up. Usually, the orbit is given in [Spacetrack](https://www.space-track.org/) or [Celestrak](https://celestrak.org/) websites, even though for small satellites launched in large batches, they can't easily identify which satellite is which, at least at the very beginning. Nevertheless, UHF comms is very forgiving and in many cases even an approximate TLE would work. For this example, we assume that a reasonably good TLE is known.

The other alternative is to start with the cartesian coordinates (for example, separation coordinates provided by the launcher)

`using_tle`

_show cartesian input and propagator, separation coordinates provided by the launcher_

In [7]:
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator

from org.orekit.propagation.numerical import NumericalPropagator
from org.hipparchus.ode.nonstiff import DormandPrince853Integrator
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel
from org.orekit.propagation import SpacecraftState
from org.orekit.orbits import CartesianOrbit, OrbitType
from org.hipparchus.geometry.euclidean.threed import Vector3D
from orekit import JArray_double

from org.orekit.utils import IERSConventions, TimeStampedPVCoordinates, Constants
from org.orekit.frames import FramesFactory

# True if a TLE is used for the orbit, False if a cartesian initial condition is used 
using_tle = True

if using_tle:
    # Insert your TLE here
    line1 = "1 40697U 15028A   20164.50828565  .00000010  00000-0  20594-4 0  9999"
    line2 = "2 40697  98.5692 238.8182 0001206  86.9662 273.1664 14.30818200259759"

    # Init TLE object
    tle = TLE(line1, line2)
    
    # Set up the SGP4 propagator
    propagator = TLEPropagator.selectExtrapolator(tle)
    
else:
    # TODO check this
    
     # Init useful frames
    itrf = FramesFactory.getITRF(IERSConventions.IERS_2010, True)
    gcrf = FramesFactory.getGCRF()
    teme = FramesFactory.getTEME()
    
    # Initial state with cartesian coordinates
    initialDate = AbsoluteDateExt("2020-06-12T12:11:55.88016Z", utc)
    position = Vector3D(-3740326.276351476, -6117114.980631275, 7333.322457122324)
    velocity = Vector3D(-928.5273226857054, 586.6711336453583, 7376.427434257024)
    pos_vel_time = TimeStampedPVCoordinates(initialDate, position, velocity)
    initial_orbit = CartesianOrbit(pos_vel_time, gcrf, Constants.WGS84_EARTH_MU)
    
    satellite_mass = 100.0  # spacecraft mass, unit kg.
    initialState = SpacecraftState(initial_orbit, satellite_mass) 

    # Set up the numerical propagator
    minStep = 0.001
    maxstep = 1000.0
    initStep = 60.0
    
    # spatial tolerance (meters)
    positionTolerance = 1.0 
    
    tolerances = NumericalPropagator.tolerances(positionTolerance, 
                                            initial_orbit, 
                                            initial_orbit.getType())

    integrator = DormandPrince853Integrator(minStep, maxstep, 
    JArray_double.cast_(tolerances[0]),  # Double array of doubles needs to be casted in Python
    JArray_double.cast_(tolerances[1]))
    integrator.setInitialStepSize(initStep)

    propagator = NumericalPropagator(integrator)
    propagator.setOrbitType(OrbitType.CARTESIAN)
    propagator.setInitialState(initialState)

    gravityProvider = GravityFieldFactory.getNormalizedProvider(10, 10)
    propagator.addForceModel(HolmesFeatherstoneAttractionModel(FramesFactory.getITRF(IERSConventions.IERS_2010, True), gravityProvider))

    end_state = propagator.propagate(search_interval.start, search_interval.end)
    
    print(end_state)
    

398600441800000.0


After all the preparations, the analyses can finally be run. The first analysis is to generate the "satellite illumination events", showing when the satellite sees the sunlight, which is critical for thermal analyses and power generation.

This is also useful to relate the received telemetry to the physical events. For example, the telemetry should show the activation of some of the solar arrays around illumination start time - the absence of any activity may therefore indicate an attitude determination or control failure, erroneous orbit definition or a solar array failure.

In [11]:
from satkit.eventfinders.eventfinders import sat_illum_finder


# ----- find sat illumination times -----
illum_times = sat_illum_finder(search_interval, propagator, use_total_eclipse=True)

print("Satellite Illumination Intervals:")
print("----------------------------------------------------------------------------")
print(illum_times)


Satellite Illumination Intervals:
----------------------------------------------------------------------------
[ 2020-06-10T00:00:00.000Z  2020-06-10T01:02:46.53575911303543Z ]
[ 2020-06-10T01:36:53.78964055695255Z  2020-06-10T02:43:28.39400976578168Z ]
[ 2020-06-10T03:17:35.64025448331481Z  2020-06-10T04:24:10.25279842864484Z ]
[ 2020-06-10T04:58:17.49131853725282Z  2020-06-10T06:04:52.11212533419479Z ]
[ 2020-06-10T06:38:59.3428337338965Z  2020-06-10T07:45:33.97199032249657Z ]
[ 2020-06-10T08:19:41.1948013995225Z  2020-06-10T09:26:15.83239292163873Z ]
[ 2020-06-10T10:00:23.04722296823704Z  2020-06-10T11:06:57.69333248314624Z ]
[ 2020-06-10T11:41:04.90009977224193Z  2020-06-10T12:47:39.55480833853304Z ]
[ 2020-06-10T13:21:46.75343283812836Z  2020-06-10T14:28:21.41681997751485Z ]
[ 2020-06-10T15:02:28.60722275719823Z  2020-06-10T16:09:03.27936718069881Z ]
[ 2020-06-10T16:43:10.46146961963413Z  2020-06-10T17:49:45.14245011115915Z ]
[ 2020-06-10T18:23:52.31617305832791Z  2020-06-10T19:30

In [12]:
from satkit.eventfinders.eventfinders import gnd_pass_finder

# ----- find passes ----- 

# elevation definition
elevation = 5 * u.deg
    
passes = gnd_pass_finder(
    search_interval,
    gnd_station,
    elevation,
    propagator=propagator,
)

print(f"Groundstation Contact Intervals (above {elevation:~P} elevation):")
print("----------------------------------------------------------------------------")
print(passes)

Groundstation Contact Intervals (above 5 deg elevation):
----------------------------------------------------------------------------
[ 2020-06-10T07:03:15.36455811220952Z  2020-06-10T07:14:44.37948488083933Z ]
[ 2020-06-10T08:42:41.71425483294207Z  2020-06-10T08:54:06.80962669148074Z ]
[ 2020-06-10T18:20:19.30156820664889Z  2020-06-10T18:32:16.44348785874871Z ]
[ 2020-06-10T20:00:34.29551820712763Z  2020-06-10T20:11:14.73155280207742Z ]



Add az-el for all passes.